## Criando tabelas
***

Então já sabe como criar tabelas em SQL, agora pode utilizar o SQLAlchemy para as criar. Se não estiver familiarizado com a programação orientada a objectos (OOP) nesta altura, terá de a aprender primeiro, uma vez que iremos utilizar o Mapeamento Objeto-Relacional (ORM) para criar tabelas.

O SQLAlchemy é um poderoso Mapeador Objeto-Relacional (ORM) para Python, que permite que você interaja com seu banco de dados usando objetos e classes Python em vez de escrever consultas SQL brutas. O ORM fornece uma camada de abstração de alto nível sobre o SQL, facilitando o trabalho com bancos de dados de uma maneira mais pythonica, enquanto ainda aproveita todo o poder do SQL.

Basicamente, você terá acesso às tabelas da sua base de dados como se fossem objectos Python, e às colunas das tabelas como se fossem atributos desses objectos.

A ideia central por detrás do ORM é mapear as tabelas da base de dados para classes Python, e as linhas das tabelas para instâncias de classes (objetos). Desta forma, será possível efetuar operações na base de dados utilizando conceitos de programação orientada a objetos, como a herança, as associações e o encapsulamento.

In [1]:
from sqlalchemy import create_engine, URL
from sqlalchemy.orm import sessionmaker
url = URL.create(
    drivername="postgresql+psycopg2",  # driver name = postgresql + the library we are using (psycopg2)
    username='notebook',
    password='notebook',
    host='postgres',
    database='notebook',
    port=5432
)
engine = create_engine(url, pool_size=10, max_overflow=20, pool_recycle=3600)
Session = sessionmaker(bind=engine)

Para mapear nossas classes Python para tabelas de banco de dados, usaremos o sistema Declarativo do SQLAlchemy.

Para começar a criar tabelas, é necessário herdar uma classe base específica do SQLAlchemy, para que o SQLAlchemy saiba como mapear os resultados das suas consultas para objectos Python.

Essa classe é chamada de base declarativa e é criada assim:

```py
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass
```

Agora podes começar a criar tabelas como classes Python.

Lembra-se de como criámos a tabela de utilizadores em SQL?

```sql
CREATE TABLE users IF NOT EXISTS (
    telegram_id   BIGINT PRIMARY KEY,
    full_name     VARCHAR(255) NOT NULL,
    username      VARCHAR(255),
    language_code VARCHAR(255) NOT NULL,
    created_at    TIMESTAMP DEFAULT NOW(),
    referrer_id   BIGINT,
    FOREIGN KEY (referrer_id)
        REFERENCES users (telegram_id)
        ON DELETE SET NULL
);
```

Vamos seguir o seguinte passo a passo:

1. Para **criar uma tabela** no SQLAlchemy, é necessário criar uma classe que herda da base declarativa.

2. Para **criar colunas** na tabela, é necessário criar novos atributos e atribuir-lhes a classe `Column`. Desde a versão 2.0, é possível utilizar a função `mapped_column` para criar colunas e as anotações de tipo mapeado para definir os tipos de colunas.

3. Para utilizar **tipos de dados SQL**, tem de importar objectos específicos do módulo `sqlalchemy`. Estes objectos têm normalmente os mesmos nomes. Exemplos: `BIGINT`, `VARCHAR`, `TIMESTAMP`.

4. Para criar uma **chave primária**, é necessário passar o argumento `primary_key` para a coluna.

5. Para criar uma **restrição não nula**, é necessário passar o argumento `nullable` para a coluna.

6. Para criar um **valor default**, é necessário passar o argumento `server_default` à coluna.

7. Para criar uma **chave estrangeira**, é necessário passar o argumento `ForeignKey` para a coluna e preencher os seus argumentos.

8. Para utilizar expressões SQL do SQLAlchemy, pode utilizar o módulo `sqlalchemy.sql.expression`. Por exemplo, pode utilizar `null()` ou `false()` para criar um valor padrão NULL ou FALSE para uma coluna.

9. Para utilizar **funções do SQL** no SQLAlchemy, pode utilizar o módulo `sqlalchemy.func`. Por exemplo, pode utilizar `func.now()` para criar um valor NOW() por defeito para uma coluna.

10. Para **dar um nome a uma tabela**, é necessário especificar sempre o atributo `__tablename__`.

In [2]:
from sqlalchemy import INTEGER
from sqlalchemy.orm import DeclarativeBase


class Base(DeclarativeBase):
    pass

Depois disso, declaramos as colunas que compõem cada tabela.

Estas colunas são declaradas utilizando uma anotação de tipagem especial chamada `Mapped`. O tipo de dados Python associado a cada anotação Mapped determina o tipo de dados SQL correspondente, por exemplo, `int` para `INTEGER` ou `str` para `VARCHAR`. A **nulidade** é baseada na utilização ou não do modificador de tipo `Optional[]`, mas também pode ser especificada explicitamente utilizando o `parâmetro nullable`.

A diretiva `mapped_column()` é aplicada a atributos baseados em colunas, permitindo que o SQLAlchemy manipule propriedades de coluna, como valores padrão, restrições de chave primária e restrições de chave estrangeira. Toda classe mapeada pelo ORM deve ter pelo menos uma coluna declarada como chave primária. No nosso exemplo, `User.telegram_id` é marcado como a chave primária definindo `primary_key=True`.

In [3]:
import datetime
from typing import Optional
from sqlalchemy.dialects.postgresql import TIMESTAMP
from sqlalchemy.sql.functions import func
from sqlalchemy.orm import Mapped, mapped_column
from sqlalchemy import ForeignKey, BIGINT, VARCHAR, String

class TimestampMixin:
    created_at: Mapped[datetime] = mapped_column(TIMESTAMP, server_default=func.now())
    updated_at: Mapped[datetime] = mapped_column(TIMESTAMP, server_default=func.now(), onupdate=func.now())


class User(Base, TimestampMixin):
    """
    Classe de usuário no banco de dados
    """

    __tablename__ = "users"

    telegram_id: Mapped[int] = mapped_column(BIGINT, primary_key=True)
    full_name: Mapped[str] = mapped_column(VARCHAR(255))
    username: Mapped[Optional[str]] = mapped_column(VARCHAR(255), nullable=True)
    language_code: Mapped[str] = mapped_column(VARCHAR(255))
    created_at: Mapped[datetime] = mapped_column(TIMESTAMP, server_default=func.now())
    referrer_id: Mapped[Optional[int]] = mapped_column(BIGINT, ForeignKey('users.telegram_id', ondelete='SET NULL'))

Vamos criar as outras tabelas

In [4]:
class Orders(Base, TimestampMixin):
    """
    Tabela de pedidos
    """

    __tablename__ = "orders"

    order_id: Mapped[int] = mapped_column(INTEGER, primary_key=True)
    user_id: Mapped[int] = mapped_column(BIGINT, ForeignKey("users.telegram_id", ondelete="CASCADE"))

```sql
CREATE TABLE orders (
    order_id   SERIAL PRIMARY KEY,
    user_id    BIGINT NOT NULL,
    created_at TIMESTAMP DEFAULT NOW(),
    FOREIGN KEY (user_id)
        REFERENCES users (telegram_id)
        ON DELETE CASCADE
);
```

In [5]:
class Products(Base, TimestampMixin):
    """
    Tabela de produtos
    """
    
    __tablename__ = "products"

    product_id: Mapped[int] = mapped_column(INTEGER, primary_key=True)
    title: Mapped[str] = mapped_column(String(255))
    description: Mapped[str]

```sql
CREATE TABLE products (
    product_id  SERIAL PRIMARY KEY,
    title       VARCHAR(255) NOT NULL,
    description TEXT,
    created_at  TIMESTAMP DEFAULT NOW()
);
```

In [6]:
class OrderProducts(Base):
    """
    Cria a tabela estrangeira do relacionamento entre pedido e produtos
    """
    
    __tablename__ = "order-products"

    order_id: Mapped[int] = mapped_column(INTEGER, ForeignKey("orders.order_id", ondelete="CASCADE"), primary_key=True)
    product_id: Mapped[int] = mapped_column(INTEGER, ForeignKey("products.product_id", ondelete="RESTRICT"), primary_key=True)
    quantity: Mapped[int]

```sql
CREATE TABLE order_products (
    order_id   INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity   INTEGER NOT NULL,
    FOREIGN KEY (order_id)
        REFERENCES orders (order_id)
        ON DELETE CASCADE,
    FOREIGN KEY (product_id)
        REFERENCES products (product_id)
        ON DELETE RESTRICT
);
```

Acabámos de definir as tabelas, mas ainda não as criámos na base de dados.

Embora NÃO SEJA RECOMENDADO, uma vez que gostariamos de acompanhar as alterações na base de dados, e acompanhar as alterações com o SQLAlchemy resume-se a escrevê-las como instruções SQL, o que não é conveniente, gostaria de lhe mostrar como criar as suas tabelas apenas com o SQLAlchemy. Mais tarde utilizaremos o [Alembic](https://alembic.sqlalchemy.org/en/latest/) para criar as tabelas por meio de migrações.

In [7]:
# Você pode deletar todas as tabelas com esse comando
Base.metadata.drop_all(engine)

# E recria-las com esse comando
Base.metadata.create_all(engine)